In [1]:
import pandas as pd
from konlpy.tag import Okt

In [12]:
import numpy as np

In [2]:
okt = Okt()

In [3]:
# 긍정 점수 사전 읽기
positive_dict = pd.read_csv('positive_score_dict.csv')
positive_dict = positive_dict.drop_duplicates(subset='단어')
positive_dict.set_index('단어', inplace=True)

In [27]:
result2 = pd.read_csv('community_token_final2.csv')

In [28]:
result2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74750 entries, 0 to 74749
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     74750 non-null  object
 1   날짜      74750 non-null  object
 2   내용      74750 non-null  object
 3   종목      74750 non-null  object
 4   내용_토큰   72512 non-null  object
dtypes: object(5)
memory usage: 2.9+ MB


In [29]:
print(result2['내용_토큰'].isnull().sum())      # 불러온 DataFrame의 결측값 수

2238


In [30]:
result2[result2['내용_토큰'].isnull()].head()

,닉네임,날짜,내용,종목,내용_토큰
7,삼전35만원,2025-01-11T17:26:18+09:00,https://youtu.be/f-8vau3NfMY?si=-dLlQD9Ph5OU1Wes,5930,NaN
23,세쓰봉,2025-01-11T12:12:07+09:00,…,5930,NaN
42,우엉차우엉차,2025-01-10T21:35:48+09:00,켁,5930,NaN
52,씁쓸하구만,2025-01-10T18:03:34+09:00,.,5930,NaN
54,으아아아이이이,2025-01-10T17:46:24+09:00,ㅎㅇㅌ,5930,NaN


In [31]:
# NaN 값인 행 제거
result2 = result2.dropna(subset=['내용_토큰'])

# 인덱스 재설정
result2.reset_index(drop=True, inplace=True)

In [32]:
result2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72512 entries, 0 to 74749
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     72512 non-null  object
 1   날짜      72512 non-null  object
 2   내용      72512 non-null  object
 3   종목      72512 non-null  object
 4   내용_토큰   72512 non-null  object
dtypes: object(5)
memory usage: 3.3+ MB


In [34]:
result2.head()

,닉네임,날짜,내용,종목,내용_토큰
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,디자인 변화 울트라 기존 디자인 에서 둥글다 모서리 변경 되어다 카메라 모듈 디자인...
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,만원 만원 물리다 대부분 이라 만원 까지 오르다 물리다 분들 원금 회복 다시 매도 ...
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,분할 매다 좋다 타점 이지만 너무 느리다 오래 걸리다 살바 너무 많다 단점
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,존버하 오르다
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,오르다 겁니다 오늘 열심히 하고 퇴근 하다


In [5]:
# 각 열에 대해 okt.morphs(stem=True) 적용 및 문자열 변환
result2['형용사_어간'] = result2['내용_형용사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['부사_어간'] = result2['내용_부사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['동사_어간'] = result2['내용_동사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(result2)  # 초기화

for idx in range(len(result2)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_토큰']:
        word_list = result2.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(' ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    result2['label'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]

except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(result2)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(result2[['label']])

In [38]:
result2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72512 entries, 0 to 72511
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     72512 non-null  object
 1   날짜      72512 non-null  object
 2   내용      72512 non-null  object
 3   종목      72512 non-null  object
 4   내용_토큰   72512 non-null  object
 5   label   72512 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 3.3+ MB


In [43]:
result2.head()

,닉네임,날짜,내용,종목,내용_토큰,label
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,005930,디자인 변화 울트라 기존 디자인 에서 둥글다 모서리 변경 되어다 카메라 모듈 디자인...,1
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,005930,만원 만원 물리다 대부분 이라 만원 까지 오르다 물리다 분들 원금 회복 다시 매도 ...,0
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,005930,분할 매다 좋다 타점 이지만 너무 느리다 오래 걸리다 살바 너무 많다 단점,1
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,005930,존버하 오르다,0
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,005930,오르다 겁니다 오늘 열심히 하고 퇴근 하다,1


In [40]:
# 라벨 카운팅
result2['label'].value_counts()

label
 0    44198
-1    15335
 1    12979
Name: count, dtype: int64

In [44]:
# 라벨 추가된 df 파일 저장
result2.to_csv('community_add_poslabel.csv', index=False, encoding='utf-8-sig')